In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import scipy
import os
import nept

In [ ]:
thisdir = os.getcwd()
# data_filepath = os.path.join(thisdir, "cache", "data")
output_filepath = os.path.join(thisdir, "plots", "tmaze")
if not os.path.exists(output_filepath):
    os.makedirs(output_filepath)

In [ ]:
date = "2019-05-18"
data_filepath = os.path.join("E:\\", "data", "data-experience", "R181", "RR3", "R181_"+date+"_recording")

In [ ]:
data_filepath

In [ ]:
import info.r181_test as info

In [ ]:
def get_trials(events):    
    
    starts = events["trial_start"]
    stops = events["trial_end"]
    
    if len(starts) < len(stops):
        stops = stops[1:]
    if len(starts) > len(stops):
        starts = starts[:-1]

    return nept.Epoch(starts, stops)

In [ ]:
event_filename = date+"_Events.nev"
events = nept.load_events(os.path.join(data_filepath, event_filename), info.event_labels)
trials = get_trials(events)
print("Mean:", np.mean(trials.durations), "(s) +/- sem:", scipy.stats.sem(trials.durations))

In [ ]:
spikes = nept.load_spikes(data_filepath)

In [ ]:
lfp = nept.load_lfp(os.path.join(data_filepath, info.lfp_swr_filename))

In [ ]:
plt.plot(lfp.time, lfp.data)
plt.show()

In [ ]:
plt.plot(lfp.time, lfp.data)
plt.xlim(89757, 91000)
plt.show()

In [ ]:
position = nept.load_position(os.path.join(data_filepath, info.position_filename), pxl_to_cm=[1, 1])

In [ ]:
plt.plot(position.x, position.y, ".")
plt.show()